### 数据库操作
从数据库中导入POI数据

In [8]:
import psycopg2

con = psycopg2.connect(database="prato", user="postgres", password="1218", host="127.0.0.1", port="5432")

cur = con.cursor()
cur.execute("select * from function")
rows = cur.fetchall()

print("Database opened successfully")

Database opened successfully


### GeoMath
提供坐标与经纬度之间的相互转换

In [7]:
# 基本静态变量
center = [43.878150000000005, 11.083850000000002]
equatorial_radius = 6378137.0
polar_radius = 6356752.3
earth_radius = 6367908.065252312
map_ratio = 0.7211005671900231

In [5]:
# 方法
import math

def calcEarthRadius(lat):
    global earth_radius
    global equatorial_radius
    global polar_radius
    lat = math.radians(lat)
    
    a = equatorial_radius
    b = polar_radius

    ta = a * math.cos(lat)
    tb = b * math.sin(lat)

    earth_radius = math.sqrt((ta * a * ta * a + tb * b * tb * b) / (ta * ta + tb * tb))


def haversineDistance(lat1, lng1, lat2, lng2):
    calcEarthRadius((lat1+lat2)/2)

    dlat = math.radians(lat2 - lat1)
    dlng = math.radians(lng2 - lng1)

    a = math.sin(dlat/2)*math.sin(dlat/2) + math.sin(dlng/2)*math.sin(dlng/2)*math.cos(lat1)*math.cos(lat2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))


def latLngToXY(lat, lng):
    global map_ratio
    dist = haversineDistance(lat, lng, center[0], center[1])
    y = math.radians(lat - center[0])
    xp = math.radians(lng - center[1])
    
    if(map_ratio == 0.0):
        yp = math.log(math.tan(math.pi / 4.0 + math.radians(lat) / 2.0)) - math.log(math.tan(math.pi/4.0 + math.radians(center[0]) / 2.0))
        map_ratio = y/yp

    x = map_ratio * xp
    
    return earth_radius * x, earth_radius * y
    

def xyToLatLng(x, y):
    global map_ratio
    lat = math.degrees(y / earth_radius) + center[0]
    x /= map_ratio
    lng = math.degrees(x / earth_radius) + center[1]

    return lat, lng


calcEarthRadius(center[0])

#### Test function
检查经纬度转换是否正确

In [6]:
print(earth_radius)
x, y = latLngToXY(43.831, 11.0202)
print(x, y)
x, y = latLngToXY(43.9253, 11.1475)
print(x, y)

6367908.065252312
-5101.160603307125 -5240.303600544768
5101.146521771683 5240.289134910398


In [11]:
from time import time

start = time()

cnt = 0
for row1 in rows:
    x, y = latLngToXY(row1[1], row1[2])
    
    cnt += 1
    print("{}/{}".format(cnt, len(rows)))
    print("time cost: ", time() - start)
    for row2 in rows:
        x, y = latLngToXY(row2[1], row2[2])



496169090271
695/23925
time cost:  75.64096283912659
696/23925
time cost:  75.74496293067932
697/23925
time cost:  75.84796619415283
698/23925
time cost:  75.95496153831482
699/23925
time cost:  76.05696129798889
700/23925
time cost:  76.16196155548096
701/23925
time cost:  76.2579619884491
702/23925
time cost:  76.3569598197937
703/23925
time cost:  76.46296429634094
704/23925
time cost:  76.56096267700195
705/23925
time cost:  76.66296601295471
706/23925
time cost:  76.76696681976318
707/23925
time cost:  76.87596201896667
708/23925
time cost:  76.97996234893799
709/23925
time cost:  77.0779640674591
710/23925
time cost:  77.17595982551575
711/23925
time cost:  77.27996277809143
712/23925
time cost:  77.38696074485779
713/23925
time cost:  77.4829614162445
714/23925
time cost:  77.58196210861206
715/23925
time cost:  77.67996597290039
716/23925
time cost:  77.77996158599854
717/23925
time cost:  77.88296175003052
718/23925
time cost:  77.98096060752869
719/23925
time cost:  78.075961

KeyboardInterrupt: 

In [12]:
import numpy as np
from scipy import spatial
x, y = np.mgrid[0:5, 2:8]

In [14]:
x.ravel()

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 4, 4, 4, 4, 4, 4])